In [106]:
import pandas as pd
import numpy as np

## 대여소가 새로 생기고 없어지는게 정말 많다.
## 9월 이용내역을 12월 대여소 기준으로 만드려고 해도 문제가 생긴다.

In [107]:
# # ~22년 5월 대여소 정보
# bike_station_df_2205 = pd.read_csv('D:/git_local_repository/data_for_project/seoul_bike/bike_station_2022.05.csv',encoding='CP949')

# ~21년 12월 대여소 정보
bike_station_df_2112 = pd.read_csv('D:/git_local_repository/data_for_project/seoul_bike/bike_station_2021.12.csv',parse_dates=['설치일'])

# 21.9월 대여 정보
seoul_bike_df = pd.read_csv('D:\git_local_repository\data_for_project\seoul_bike\seoul_bike_2021.09.csv',encoding='CP949')

In [108]:
seoul_bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3645807 entries, 0 to 3645806
Data columns (total 11 columns):
 #   Column    Dtype  
---  ------    -----  
 0   자전거번호     object 
 1   대여일시      object 
 2   대여 대여소번호  int64  
 3   대여 대여소명   object 
 4   대여거치대     int64  
 5   반납일시      object 
 6   반납대여소번호   int64  
 7   반납대여소명    object 
 8   반납거치대     int64  
 9   이용시간      int64  
 10  이용거리      float64
dtypes: float64(1), int64(5), object(5)
memory usage: 306.0+ MB


In [109]:
### 정리 순서
# 대여소 주소, 대여소 번호 통일
# 자전거 번호 정리
# 대여일시 2021 제거, 날짜, 시간 구분
# 불필요한 column제거 대여거치대, 반납거치대, 반납일시
# 불필요한 row 제거 1.이용거리 기준보고 outlier 제거 2. 대여일시 3월 31일 제거


In [110]:
### 안쓰는 code
# bike_station_df_2205.rename(columns={'대여소_ID' : 'id'},inplace=True)
# bike_station_df_2205['id'] = bike_id = bike_station_df_2205['id'].str.replace('ST-','').astype('int16')
# print(f'bike_station_22년 05월까지 : {len(bike_station_df_2205)}개 대여소')


# 대여소 id 양식 통일
bike_station_df_2112.rename(columns={'대여소 번호' : 'id'},inplace=True)
seoul_bike_df.rename(columns={'대여 대여소번호' : 'id' , '대여 대여소명':'주소2', '반납대여소번호' : 'id2'},inplace=True)

bike_station_df_2112['id'] = bike_station_df_2112['id'].astype('int16')


In [111]:
from datetime import datetime
BM = bike_station_df_2112['설치일'] < datetime(2021, 10, 1)


print(f'bike_station 21년 9월 30일까지 대여소 : {len(bike_station_df_2112[BM])}개 대여소')
print(f'seoul_bike_df["id"] : {len(seoul_bike_df["id"].unique())}개 대여소')
print(f'seoul_bike_df["주소2"] : {len(seoul_bike_df["주소2"].unique())}개 대여소')

bike_station 21년 9월 30일까지 대여소 : 2515개 대여소
seoul_bike_df["id"] : 2462개 대여소
seoul_bike_df["주소2"] : 2458개 대여소


In [112]:
# 자전거 번호 정리
seoul_bike_df['자전거번호'] = seoul_bike_df['자전거번호'].str.replace('SPB-','').astype('int32')
# 23.1mb => 5.3mb 감축

#  대여일시 정리
rental_date = pd.to_datetime(seoul_bike_df['대여일시'])

# 대여 일, 대여 시간 
seoul_bike_df['day'] = rental_date.dt.day.astype('int8')
seoul_bike_df['time'] = rental_date.dt.time

# 요일 설정
seoul_bike_df['weekday'] =rental_date.dt.weekday.astype('int8')

# type 변경
seoul_bike_df['id']  = seoul_bike_df['id'].astype('int16')
seoul_bike_df['id2']  = seoul_bike_df['id2'].astype('int16')
seoul_bike_df['이용시간']  = seoul_bike_df['이용시간'].astype('int8')

# 불필요한 column 드랍
seoul_bike_df.drop(columns=['대여일시','대여거치대','반납일시','반납대여소명','반납거치대'],inplace=True)

In [113]:
# 21년 9월에 사용된 bike station 추출
BM_2 = bike_station_df_2112['id'].isin(seoul_bike_df['id'])
bs_2109=bike_station_df_2112[BM & BM_2]


seoul_bike = seoul_bike_df[seoul_bike_df['id'].isin(bs_2109['id'])]
seoul_bike.info()

# memory usage: 306.0+ MB to 147.3 MB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3591134 entries, 20 to 3645585
Data columns (total 9 columns):
 #   Column   Dtype  
---  ------   -----  
 0   자전거번호    int32  
 1   id       int16  
 2   주소2      object 
 3   id2      int16  
 4   이용시간     int8   
 5   이용거리     float64
 6   day      int8   
 7   time     object 
 8   weekday  int8   
dtypes: float64(1), int16(2), int32(1), int8(3), object(2)
memory usage: 147.3+ MB


In [114]:
# id 2420개 주소2 2416개
print(f'seoul_bike["id"] : {len(seoul_bike["id"].unique())}개 대여소')
print(f'seoul_bike["주소2"] : {len(seoul_bike["주소2"].unique())}개 대여소')


a = seoul_bike.groupby(['id','주소2']).size()
b = [a.index[i][1] for i in range(len(a))]
pd.Series(b).value_counts()


# 4개 주소 중복
# 이중 한양수자인아파트 앞 id 4811은 사용하지 않음
# 국회3문, 천왕이펜하우스5단지 앞은 완전 같음
# 신봉천주유소는 A,B로 구분
# 쓰다써..

seoul_bike["id"] : 2420개 대여소
seoul_bike["주소2"] : 2416개 대여소


한양수자인아파트 앞         2
국회3문               2
천왕이펜하우스5단지 앞       2
신봉천주유소             2
국립국악중,고교 정문 맞은편    1
                  ..
등촌2동주민센터           1
용문사 버스정류장          1
목동사거리 버스정류장        1
곰달래 문화복지센터 1-1     1
잠실나들목6             1
Length: 2416, dtype: int64

In [116]:
seoul_bike

,자전거번호,id,주소2,id2,이용시간,이용거리,day,time,weekday
20,45605,102,망원역 1번출구 앞,4213,7,0.00,1,02:34:32,2
21,42126,102,망원역 1번출구 앞,107,8,814.13,1,05:53:58,2
22,55711,102,망원역 1번출구 앞,107,6,820.00,1,07:41:36,2
23,51047,102,망원역 1번출구 앞,102,9,0.00,1,07:45:30,2
24,50155,102,망원역 1번출구 앞,3010,7,0.00,1,07:57:11,2
...,...,...,...,...,...,...,...,...,...
3645581,52694,4869,잠실나들목6,1249,16,1900.00,30,23:00:56,3
3645582,45908,4869,잠실나들목6,1269,30,4590.00,30,22:56:46,3
3645583,43746,4869,잠실나들목6,1002,32,7077.05,30,23:20:41,3
3645584,36631,4869,잠실나들목6,2550,54,9738.93,30,23:22:48,3


In [126]:
seoul_bike.drop(columns='주소2')
seoul_bike['id1_long'] = np.zeros(len(seoul_bike))
seoul_bike['id1_lat'] = np.zeros(len(seoul_bike))
seoul_bike['id2_long'] = np.zeros(len(seoul_bike))
seoul_bike['id2_lat'] = np.zeros(len(seoul_bike))
seoul_bike = seoul_bike[['day','time','weekday','자전거번호','id','id2','이용시간','이용거리','id1_long','id1_lat','id2_long','id2_lat']]
seoul_bike.reset_index(drop=True,inplace=True)
seoul_bike.columns=['day','time','weekday','by_id','st_id1','st_id2','riding_time','dist','id1_long','id1_lat','id2_long','id2_lat']
seoul_bike

,day,time,weekday,by_id,st_id1,st_id2,riding_time,dist,id1_long,id1_lat,id2_long,id2_lat
0,1,02:34:32,2,45605,102,4213,7,0.00,0.0,0.0,0.0,0.0
1,1,05:53:58,2,42126,102,107,8,814.13,0.0,0.0,0.0,0.0
2,1,07:41:36,2,55711,102,107,6,820.00,0.0,0.0,0.0,0.0
3,1,07:45:30,2,51047,102,102,9,0.00,0.0,0.0,0.0,0.0
4,1,07:57:11,2,50155,102,3010,7,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3591129,30,23:00:56,3,52694,4869,1249,16,1900.00,0.0,0.0,0.0,0.0
3591130,30,22:56:46,3,45908,4869,1269,30,4590.00,0.0,0.0,0.0,0.0
3591131,30,23:20:41,3,43746,4869,1002,32,7077.05,0.0,0.0,0.0,0.0
3591132,30,23:22:48,3,36631,4869,2550,54,9738.93,0.0,0.0,0.0,0.0


불필요한 id 제거
<br>

반납대여소번호	반납대여소명
<br>
3	중랑센터
<br>
9992	사대문센터
<br>
9998	영남주차장 정비센터
<br>
9980	에이텍
<br>
5	상암센터 정비실
<br>
9999	상담센터
<br>
9997	강남센터
<br>
9990	_도봉센터

In [130]:
del_ids= np.array([  9992,   9998, -31073,   9980,      5,   9999,      3,   9997,  9990], dtype='int16')
seoul_bike = seoul_bike[~(seoul_bike['st_id2'].isin(del_ids))]


,day,time,weekday,by_id,st_id1,st_id2,riding_time,dist,id1_long,id1_lat,id2_long,id2_lat
0,1,02:34:32,2,45605,102,4213,7,0.00,0.0,0.0,37.564156,126.908150
1,1,05:53:58,2,42126,102,107,8,814.13,0.0,0.0,37.557510,126.918503
2,1,07:41:36,2,55711,102,107,6,820.00,0.0,0.0,37.557510,126.918503
3,1,07:45:30,2,51047,102,102,9,0.00,0.0,0.0,37.555649,126.910629
4,1,07:57:11,2,50155,102,3010,7,0.00,0.0,0.0,37.558296,126.925385
...,...,...,...,...,...,...,...,...,...,...,...,...
3591129,30,23:00:56,3,52694,4869,1249,16,1900.00,0.0,0.0,37.505463,127.077347
3591130,30,22:56:46,3,45908,4869,1269,30,4590.00,0.0,0.0,37.511944,127.091217
3591131,30,23:20:41,3,43746,4869,1002,32,7077.05,0.0,0.0,37.545261,127.125946
3591132,30,23:22:48,3,36631,4869,2550,54,9738.93,0.0,0.0,37.467384,127.041458


In [ ]:
### st_id1에 맞는 위도 경도 넣기
for id in seoul_bike["st_id1"].unique() :
    seoul_bike.loc[(seoul_bike['st_id1']==id),'id2_long'] = bs_2109.query('id==@id')['위도'].values[0]
    seoul_bike.loc[(seoul_bike['st_id1']==id),'id2_lat'] = bs_2109.query('id==@id')['경도'].values[0]


### st_id2에 맞는 위도 경도 넣기
bike_station_df_2106 = pd.read_csv('D:/git_local_repository/data_for_project/seoul_bike/bike_station_2021.06.csv')

id_2106 = seoul_bike['st_id2'].isin(bike_station_df_2106['id'])

## bike_station_df_2106에 있는 위도 경도 st_id2에 넣기
for id in seoul_bike[id_2106]['st_id2'].unique() :
    seoul_bike.loc[(seoul_bike['st_id2']==id),'id2_long'] = bike_station_df_2106.query('id==@id')['위도'].values[0]
    seoul_bike.loc[(seoul_bike['st_id2']==id),'id2_lat'] = bike_station_df_2106.query('id==@id')['경도'].values[0]

## bike_station_2106에 없는 위도 경도 bike_station_df_2112에서 찾기
for id in seoul_bike[~id_2106]['st_id2'].unique() :
    seoul_bike.loc[(seoul_bike['st_id2']==id),'id2_long'] = bs_2109.query('id==@id')['위도'].values[0]
    seoul_bike.loc[(seoul_bike['st_id2']==id),'id2_lat'] = bs_2109.query('id==@id')['경도'].values[0]


In [132]:
seoul_bike
### 자료 저장

# seoul_bike.to_hdf('D:\git_local_repository\data_for_project\seoul_bike\seoul_bike_2021_09.hdf5',key='seoul_bike')

,day,time,weekday,by_id,st_id1,st_id2,riding_time,dist,id1_long,id1_lat,id2_long,id2_lat
0,1,02:34:32,2,45605,102,4213,7,0.00,0.0,0.0,37.564156,126.908150
1,1,05:53:58,2,42126,102,107,8,814.13,0.0,0.0,37.557510,126.918503
2,1,07:41:36,2,55711,102,107,6,820.00,0.0,0.0,37.557510,126.918503
3,1,07:45:30,2,51047,102,102,9,0.00,0.0,0.0,37.555649,126.910629
4,1,07:57:11,2,50155,102,3010,7,0.00,0.0,0.0,37.558296,126.925385
...,...,...,...,...,...,...,...,...,...,...,...,...
3591129,30,23:00:56,3,52694,4869,1249,16,1900.00,0.0,0.0,37.505463,127.077347
3591130,30,22:56:46,3,45908,4869,1269,30,4590.00,0.0,0.0,37.511944,127.091217
3591131,30,23:20:41,3,43746,4869,1002,32,7077.05,0.0,0.0,37.545261,127.125946
3591132,30,23:22:48,3,36631,4869,2550,54,9738.93,0.0,0.0,37.467384,127.041458
